In [65]:
import pandas as pd
import numpy as np
import time
import os
from os import listdir
from os.path import isfile, join, basename

import sys

from lag_features import *
from other_functions import *

import datetime
from sklearn.preprocessing import LabelEncoder

In [66]:
DIR = '/Users/carlosperezricardo/Desktop/TFM'

CARRIER = 19393
years_to_load = ['2017','2018','2019']

In [67]:
folder = os.path.join(DIR, 'datasets')
files = [f for f in listdir(folder) if isfile(join(folder, f))]
files = sorted(files)

In [68]:
df = pd.DataFrame()
for file in files:
    loading = []
    if '.zip' in file:
        for year in years_to_load:
            loading.append(year in file)
        output = any(loading)
        if output:
            add_df = pd.read_csv(os.path.join(DIR,'datasets',file), parse_dates=['FL_DATE'])
            add_df = add_df[add_df.OP_CARRIER_AIRLINE_ID == CARRIER]
            df = pd.concat([df, add_df], axis=0)
df.reset_index(drop=True, inplace=True)

In [69]:
df.head()

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,...,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 29
0,2017-10-01,WN,19393,WN,N789SW,2937,10140,1014003,30140,ABQ,...,1204.0,-11.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,2017-10-01,WN,19393,WN,N7825A,2736,10140,1014003,30140,ABQ,...,1532.0,-8.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,2017-10-01,WN,19393,WN,N464WN,5108,10140,1014003,30140,ABQ,...,2113.0,-22.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,2017-10-01,WN,19393,WN,N462WN,2934,10140,1014003,30140,ABQ,...,923.0,-12.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,2017-10-01,WN,19393,WN,N7878A,3315,10140,1014003,30140,ABQ,...,2049.0,4.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [70]:
ini_date = datetime.datetime(2017, 10, 1)

train_ini_date = datetime.datetime(2018, 1, 1)
train_fin_date = datetime.datetime(2019, 9, 1)

test_ini_date = datetime.datetime(2019, 9, 1)
test_fin_date = datetime.datetime(2019, 9, 30)

In [71]:
df = df[(df.FL_DATE > ini_date) & (df.FL_DATE <= test_fin_date)]

In [72]:
df = df[df.OP_CARRIER_AIRLINE_ID == CARRIER]

In [73]:
df.shape

(2592098, 30)

In [74]:
# Feature Generation 
df = date_features(df, 'FL_DATE')

In [75]:
df.groupby('TAIL_NUM')['ARR_DELAY'].apply(lambda x: x.shift(1))

3505        NaN
3506        NaN
3507        NaN
3508        NaN
3509        NaN
           ... 
2595598   -13.0
2595599     2.0
2595600   -14.0
2595601    -8.0
2595602     1.0
Name: ARR_DELAY, Length: 2592098, dtype: float64

In [76]:
df.groupby('TAIL_NUM')['ARR_DELAY'].apply(lambda x: x.shift(1).rolling(5).apply(np.mean))

In [ ]:
df.shape

(2592098, 35)

In [ ]:
df.head()

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,...,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 29,month,day,year,year_month,weekday
3505,2017-10-02,WN,19393,WN,N8693A,938,12191,1219102,31453,HOU,...,NaN,NaN,NaN,NaN,NaN,10,2,2017,2017_010,0
3506,2017-10-02,WN,19393,WN,N8683D,4825,12191,1219102,31453,HOU,...,NaN,NaN,NaN,NaN,NaN,10,2,2017,2017_010,0
3507,2017-10-02,WN,19393,WN,N8511K,4373,12191,1219102,31453,HOU,...,NaN,NaN,NaN,NaN,NaN,10,2,2017,2017_010,0
3508,2017-10-02,WN,19393,WN,N7815L,4205,12191,1219102,31453,HOU,...,NaN,NaN,NaN,NaN,NaN,10,2,2017,2017_010,0
3509,2017-10-02,WN,19393,WN,N7720F,47,12191,1219102,31453,HOU,...,NaN,NaN,NaN,NaN,NaN,10,2,2017,2017_010,0


In [ ]:
df.columns

Index(['FL_DATE', 'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER',
       'TAIL_NUM', 'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID',
       'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID', 'ORIGIN',
       'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'DEST_AIRPORT_ID',
       'DEST_AIRPORT_SEQ_ID', 'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME',
       'DEST_STATE_ABR', 'DEP_TIME', 'DEP_DELAY', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'LATE_AIRCRAFT_DELAY', 'Unnamed: 29',
       'month', 'day', 'year', 'year_month', 'weekday'],
      dtype='object')

In [ ]:
df['OP_CARRIER_AIRLINE_ID'].value_counts()

19393    2592098
Name: OP_CARRIER_AIRLINE_ID, dtype: int64

In [ ]:
train_df = df[(df.FL_DATE > datetime.datetime(2018, 1, 1)) & (df.FL_DATE <= datetime.datetime(2019, 9, 1))]
test_df = df[(df.FL_DATE > datetime.datetime(2019, 9, 1)) & (df.FL_DATE <= datetime.datetime(2019, 9, 30))]

In [ ]:
print(train_df.shape)
print(test_df.shape)

(2266109, 35)
(106432, 35)


In [ ]:
df_ = df[df.OP_CARRIER_AIRLINE_ID == CARRIER]

In [ ]:
df_ = df_[(df_.CANCELLED != 1) & (df_.DIVERTED != 1)]

In [ ]:
# Lag features by: TAIL_NUM
calculations = {}
#calculations['calc1'] = {'gb_list':['TAIL_NUM','FL_DATE'],'target':'ARR_DELAY','shifts':[30,45,60], 'windows':[10], 'funs':['mean','std']}
#calculations['calc2'] = {'gb_list':['TAIL_NUM','FL_DATE'],'target':'ARR_DELAY','shifts':[365], 'windows':[5,10], 'funs':['mean']}

#df_ = apply_calc(df_, calculations)

In [ ]:
# Lag features by: DEST_AIRPORT_ID
calculations = {}
calculations['calc3'] = {'gb_list':['DEST_AIRPORT_ID','FL_DATE'],'target':'ARR_DELAY','shifts':[30,45,60], 'windows':[10,30], 'funs':['mean']}
calculations['calc4'] = {'gb_list':['DEST_AIRPORT_ID','FL_DATE'],'target':'ARR_DELAY','shifts':[30], 'windows':[5,10,30], 'funs':['median','std']}
calculations['calc5'] = {'gb_list':['DEST_AIRPORT_ID','FL_DATE'],'target':'ARR_DELAY','shifts':[365], 'windows':[10], 'funs':['mean','std']}

df_ = apply_calc(df_, calculations)

Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s30_r10_mean
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s45_r10_mean
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s60_r10_mean
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s30_r30_mean
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s45_r30_mean
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s60_r30_mean
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s30_r60_mean
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s45_r60_mean
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s60_r60_mean
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s30_r10_median
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s45_r10_median
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s60_r10_median
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s30_r30_median
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s45_r30_median
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s60_r30_median
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s30_r60_median
Generating ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s45_r60_media

In [ ]:
# Lag features by: ORIGIN_AIRPORT_ID
calculations = {}
calculations['calc6'] = {'gb_list':['ORIGIN_AIRPORT_ID','FL_DATE'],'target':'DEP_DELAY','shifts':[30,45,60], 'windows':[5,10,30], 'funs':['mean']}
calculations['calc7'] = {'gb_list':['DEST_AIRPORT_ID','FL_DATE'],'target':'ARR_DELAY','shifts':[30], 'windows':[5,10,30], 'funs':['median','std']}
calculations['calc8'] = {'gb_list':['ORIGIN_AIRPORT_ID','FL_DATE'],'target':'DEP_DELAY','shifts':[365], 'windows':[10], 'funs':['mean']}

df_ = apply_calc(df_, calculations)

Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s30_r10_median
Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s45_r10_median
Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s60_r10_median
Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s30_r30_median
Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s45_r30_median
Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s60_r30_median
Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s365_r10_mean
Generating DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s365_r10_std


In [ ]:
# Convert times 
# [ARR_TIME, DEP_TIME]

df_['DEP_TIME'] = df_['DEP_TIME']/100
df_['ARR_TIME'] = df_['ARR_TIME']/100

df_['DEP_TIME'] = df_['DEP_TIME'].astype(int)
df_['ARR_TIME'] = df_['ARR_TIME'].astype(int)

In [ ]:
drop_cols = ['CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'LATE_AIRCRAFT_DELAY', 'Unnamed: 29']
for col in drop_cols:
    del df_[col]

In [ ]:
keep = list(df_.select_dtypes(include=np.number).columns)

for col in df_.select_dtypes(exclude=np.number).columns:
    print(col, len(df_[col].unique()))
    if len(df_[col].unique()) < 100:
        keep.append(col)
        le = LabelEncoder()
        df_[col] = le.fit_transform(df_[col])
        df_[col] = df_[col].astype('category')

FL_DATE 698
OP_UNIQUE_CARRIER 1
OP_CARRIER 1
TAIL_NUM 754
ORIGIN 89
ORIGIN_CITY_NAME 88
ORIGIN_STATE_ABR 42
DEST 89
DEST_CITY_NAME 88
DEST_STATE_ABR 42
year_month 23


In [ ]:
keep.remove('ORIGIN_CITY_NAME')
keep.remove('DEST_CITY_NAME')

In [ ]:
train_df = df_.loc[(df_.FL_DATE > datetime.datetime(2018, 1, 1)) & (df_.FL_DATE <= datetime.datetime(2019, 9, 1)), keep]
test_df = df_.loc[(df_.FL_DATE > datetime.datetime(2019, 9, 1)) & (df_.FL_DATE <= datetime.datetime(2019, 9, 30)), keep]

In [ ]:
print(train_df.shape)
print(test_df.shape)

(2214976, 53)
(104484, 53)


In [ ]:
TARGET = 'ARR_DELAY'
drop = ['DEP_DELAY']

In [ ]:
features = [ col for col in train_df.columns if col != TARGET and col not in drop ]

X_train = train_df[features]
y_train = train_df[TARGET]

X_test = test_df[features]
y_test = test_df[TARGET]

In [ ]:
train_df.head()

,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEP_TIME,DEP_DELAY,...,DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s60_r30_median,DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s365_r10_mean,DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s365_r10_std,OP_UNIQUE_CARRIER,OP_CARRIER,ORIGIN,ORIGIN_STATE_ABR,DEST,DEST_STATE_ABR,year_month
223062,19393,992,13232,1323202,30977,14679,1467903,33570,20,50.0,...,NaN,NaN,NaN,0,0,50,11,75,3,5
223063,19393,4972,13232,1323202,30977,14679,1467903,33570,11,-2.0,...,NaN,NaN,NaN,0,0,50,11,75,3,5
223064,19393,5103,13232,1323202,30977,14679,1467903,33570,8,19.0,...,NaN,NaN,NaN,0,0,50,11,75,3,5
223065,19393,2681,13232,1323202,30977,14683,1468305,33214,16,52.0,...,NaN,NaN,NaN,0,0,50,11,76,37,5
223066,19393,5213,13232,1323202,30977,14683,1468305,33214,8,29.0,...,NaN,NaN,NaN,0,0,50,11,76,37,5


In [ ]:
import lightgbm as lgbm
reg = lgbm.LGBMRegressor(metric='rmse', n_estimators=400)

In [ ]:
reg.fit(X_train, y_train)

LGBMRegressor(metric='rmse', n_estimators=400)

In [ ]:
pd.DataFrame({'columns': X_train.columns,'feature_importance':reg.feature_importances_}).sort_values('feature_importance',ascending=False)

,columns,feature_importance
46,ORIGIN,2479
48,DEST,2435
50,year_month,1317
9,ARR_TIME,1038
8,DEP_TIME,838
1,OP_CARRIER_FL_NUM,552
36,DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s30_r10_me...,279
13,weekday,261
11,day,256
39,DEP_DELAY_ORIGIN_AIRPORT_ID_FL_DATE_s30_r30_me...,158


In [ ]:
train_df_ = train_df.copy(deep=True)
train_df_['pred'] = reg.predict(X_train)

In [ ]:
train_df_[['ARR_DELAY','pred']].head(20)

,ARR_DELAY,pred
223062,16.0,2.065564
223063,-39.0,-3.146902
223064,-4.0,-8.115967
223065,31.0,4.801552
223066,12.0,-4.506810
223067,14.0,1.845770
223068,-3.0,-4.328653
223069,-8.0,-5.811084
223070,17.0,0.157209
223071,34.0,3.883770


In [ ]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(train_df_['ARR_DELAY'], train_df_['pred'], squared=True)
rmse

694.613399598049

In [ ]:
rmse = mean_squared_error(test_df['ARR_DELAY'], reg.predict(X_test), squared=True)
rmse

478.5537633741613

In [ ]:
train_df_[['ARR_DELAY','pred']].sort_values('ARR_DELAY', ascending=False).head(20)

,ARR_DELAY,pred
1944033,809.0,200.926918
665786,698.0,319.631279
1973484,631.0,81.871650
1254786,625.0,261.045987
1527731,616.0,91.804831
991289,614.0,93.527759
711116,608.0,433.047873
1281755,595.0,33.019757
1256577,587.0,55.423046
2445308,579.0,64.997690


In [ ]:
# Feature Generation

In [ ]:
train_df.head()

,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEP_TIME,DEP_DELAY,...,ARR_DELAY_DEST_AIRPORT_ID_FL_DATE_s30_r60_median,OP_UNIQUE_CARRIER,OP_CARRIER,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST,DEST_CITY_NAME,DEST_STATE_ABR,year_month
223062,19393,992,13232,1323202,30977,14679,1467903,33570,2020.0,50.0,...,NaN,0,0,50,13,11,75,74,3,5
223063,19393,4972,13232,1323202,30977,14679,1467903,33570,1148.0,-2.0,...,NaN,0,0,50,13,11,75,74,3,5
223064,19393,5103,13232,1323202,30977,14679,1467903,33570,844.0,19.0,...,NaN,0,0,50,13,11,75,74,3,5
223065,19393,2681,13232,1323202,30977,14683,1468305,33214,1612.0,52.0,...,NaN,0,0,50,13,11,76,73,37,5
223066,19393,5213,13232,1323202,30977,14683,1468305,33214,839.0,29.0,...,NaN,0,0,50,13,11,76,73,37,5


In [ ]:
keep = list()
for col in train_df.select_dtypes(exclude=np.number).columns:
    print(col, len(train_df[col].unique()))
    if len(train_df[col].unique()) < 100:
        keep.append(col)
    #le = LabelEncoder()
    #articles_df[col] = le.fit_transform(articles_df[col])
    #articles_df[col] = articles_df[col].astype('category')

OP_UNIQUE_CARRIER 1
OP_CARRIER 1
ORIGIN 89
ORIGIN_CITY_NAME 88
ORIGIN_STATE_ABR 42
DEST 89
DEST_CITY_NAME 88
DEST_STATE_ABR 42
year_month 21


In [ ]:
train_df = train_df[keep]